In [1]:
import os
import tensorflow as tf
from transformer.transformer import Transformer
from transformer.preprocess.batch_generator import BatchGenerator

In [2]:
data_path = 'data/natsume.txt'

In [3]:
batch_generator = BatchGenerator()
batch_generator.load(data_path)

In [4]:
vocab_size = batch_generator.vocab_size

In [5]:
"""
グラフは3段階のパターンに従って構築される．
(1) inference(): 予測のためにネットワークを前進実行するグラフを作成する
(2) loss(): 損失を生成する操作をグラフに追加する
(3) training(): 勾配を計算し適用する操作をグラフに追加する
"""
# (1) の inference() に該当するフェーズ．
# transformer の推論グラフを構築して，graph に格納する
# 本コードでは，transformer クラスの build_graph() メソッド内ですでに
# (2) の loss() についても記述されているので，(1) と (2) は同時に行われる
graph = tf.Graph()
with graph.as_default():
    transformer = Transformer(
        vocab_size=vocab_size,
        hopping_num=4,
        head_num=8,
        hidden_dim=512,
        dropout_rate=0.1,
        max_length=50,
    )
    transformer.build_graph()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [6]:
save_dir = 'tmp/learning/transformer/'
log_dir = os.path.join(save_dir, 'log')
ckpt_path = os.path.join(save_dir, 'checkpoints/model.ckpt')

os.makedirs(log_dir, exist_ok=True)

In [7]:
with graph.as_default():
    # (3) の training() に該当するフェーズ
    # グローバルな訓練ステップのカウンタが入る一つの変数を用意する
    global_step = tf.train.get_or_create_global_step()
    # 学習率の placeholder を設定する
    learning_rate = tf.placeholder(dtype=tf.float32, name='learning_rate')
    # optimizer は要求された学習率による勾配を適用する
    optimizer = tf.train.AdamOptimizer(
        learning_rate=learning_rate,
        beta2=0.98,
    )
    """
    minimize() 操作は，システムの訓練可能な重みを更新し，ステップをインクリメントする．
    慣例でこの操作は train_op として知られ，Twnsorflow のセッションにより，訓練の1フル
    ステップを引き起こすために実行される．
    (本コードでは train_op は optimize_op として定義されているっぽい？)
    """
    optimize_op = optimizer.minimize(transformer.loss, global_step=global_step)

    
    # loss 関数から損失テンソルを取り，tf.summary に渡す
    summary_op = tf.summary.merge([
        tf.summary.scalar('train/loss', transformer.loss),
        tf.summary.scalar('train/acc', transformer.acc),
        tf.summary.scalar('train/learning_rate', learning_rate),
    ], name='train_summary')
    # イベントファイルに要約値を生成する．
    summary_writer = tf.summary.FileWriter(log_dir, graph)
    saver = tf.train.Saver()

In [8]:
max_step = 10000
batch_size = 128
max_learning_rate = 0.0001
warmup_step = 4000

In [9]:
def get_learning_rate(step: int) -> float:
    rate = min(step ** -0.5, step * warmup_step ** -1.5) / warmup_step ** -0.5
    return max_learning_rate * rate

In [10]:
with graph.as_default():
    # 構築準備が全て完了し，必要な操作を全て生成したら，グラフを実行するために tf.Session を生成する
    sess = tf.Session()
    # セッションが作成された直後，sess.run() を呼び出すことにより，全ての tf.Variable インスタンスは
    # 各々の初期化操作により初期化される．
    """
    sess.run() メソッドは，パラメータとして渡された操作に対応するグラフの完全なサブセットを実行する．
    この最初の呼び出しでは，init 操作は変数の初期化子のみを含む tf.group である．
    グラフの残りの部分のいずれもここでは実行されず，以下の訓練ループで実行される．
    """
    sess.run(tf.global_variables_initializer())
    step = 0

In [11]:
# セッションで変数を初期化した後，訓練を開始する．
with graph.as_default():
    """
    各ステップにおいて，以前に生成した placeholder に適合するように入力データをスライスする．
    すなわち，各ステップにおいてフィード辞書を作成する．placeholder をキーとし，対応する
    フィード・テンソルを値として，python の辞書オブジェクトを生成する．
    """
    for batch in batch_generator.get_batch(batch_size=batch_size):
        """
        feed の形式:
        {
            'transformer/encoder_input:0': array(...),
            'transformer/decoder_input:0': array(...),
            'transformer/is_training:0': True,
            <tf.Tensor 'learning_rate:0' shape=<unknown> dtype=float32>: float32
        }
        """
        feed = {
            **batch,
            learning_rate: get_learning_rate(step + 1),
        }
        """
        run の呼び出しの中で取ってくるいくつかの値を指定する: [optimize_op, ... , summary_op]
        sess.run() は指定した値の数に対応したアイテムを持つタプルを返す．
        返されるタプル内の numpy 配列に対応する，取ってくる値リスト内の各テンソルは，この訓練ステップ中の
        そのテンソルの値で満たされる．train_op (ここではoptimize_op) は出力値のない操作であり，返さ
        れるタプルの対応する要素は None であるため破棄する．
        loss テンソル等の値は訓練中にモデルが発散した場合は NaN になるため，ログのためにキャプチャする．
        NaN にならず訓練が正常に実行された場合でも，ユーザに訓練の状態を知らせるために訓練ループは100
        ステップごとに簡単なステータステキストを出力する．
        """
        _, loss, acc, step, summary = sess.run([optimize_op, transformer.loss, transformer.acc, global_step, summary_op], feed_dict=feed)
        summary_writer.add_summary(summary, step)

        if step % 500 == 0:
            print(f'{step}: loss: {loss},\t acc: {acc}')
            saver.save(sess, ckpt_path, global_step=step)

            print("encode: ", batch)
            print("decode：", pred[0])

0: loss: 8.392192840576172,	 acc: 0.0
500: loss: 6.613274097442627,	 acc: 0.12712550163269043
1000: loss: 6.026534557342529,	 acc: 0.16588884592056274
1500: loss: 5.784158706665039,	 acc: 0.19024814665317535
2000: loss: 5.5631232261657715,	 acc: 0.19194410741329193
2500: loss: 5.25501823425293,	 acc: 0.22114962339401245
Instructions for updating:
Use standard file APIs to delete files with this prefix.
3000: loss: 5.0997114181518555,	 acc: 0.22545453906059265
3500: loss: 4.994719505310059,	 acc: 0.2152164727449417
4000: loss: 4.917576789855957,	 acc: 0.22235387563705444
4500: loss: 4.568525314331055,	 acc: 0.26471972465515137
5000: loss: 4.502110004425049,	 acc: 0.25910723209381104
5500: loss: 4.357829570770264,	 acc: 0.27008309960365295
6000: loss: 4.350559234619141,	 acc: 0.26181188225746155
6500: loss: 4.233307361602783,	 acc: 0.2799823582172394
7000: loss: 4.174062252044678,	 acc: 0.27615249156951904
7500: loss: 4.130034446716309,	 acc: 0.27462437748908997
8000: loss: 4.13414478302

KeyboardInterrupt: 

In [ ]:
with graph.as_default():
    sess.close()

In [ ]:
with graph.as_default():
    sess = tf.Session()
    sess.run(tf.initialize_all_variables())
    